In [1]:
import numpy as np
import math

### Loading Image

In [2]:
# Load P1 Image
def load_image(image_url):
    header = []
    with open(image_url, 'r') as f:
        for l in range(3):
            header.append(f.readline())
        y,x = [int(n) for n in header[2].split()]
        pixels = np.zeros((x, y)) # (7,24)
        for j in range(x):
            for k in range(y):
                pixel = f.read(1)
                if pixel in ['\n', ' ']:
                    pixel = f.read(1)
                pixels[j][k] = pixel
        return header, pixels.astype('int64')

### Store image w/ transformations

In [3]:
def store_image(image_url, img_data):
    header, pixels = img_data
    with open(image_url, 'w') as f:
        [f.write(l) for l in header]
        y,x = [int(n) for n in header[2].split()]
        if transform_func == 'equalize':
            equalize([pixels, LMax])
        for j in range(x):
            for k in range(y):
                # 3th - Negative filter
                if transform_func == 'negative':
                    pixels[j][k] = (LMax - 1) - pixels[j][k]
                # 3th - Thresholding filter
                if transform_func == 'threshold':
                    pixels[j][k] = int(header[3]) if pixels[j][k] >= threshold else 0
                f.write(str(int(pixels[j][k])))
            f.write('\n')
        return 0

## Histograma é dado por
$ h(r_k) = n_k $, onde $r_k$ é a tonalidade da imagem com k = 0,1,2...L-1 e $n_k$ é o número de pixels na tonalidade k

In [ ]:
# Histogram of the picture
def h(img_data):
    LMax = int(header[3]) + 1
    H = np.zeros(LMax).astype('int64')
    for x in range(img_data.shape[0]):
        for y in range(img_data.shape[1]):
            H[img_data[x][y]] += 1
    return H

### Normalização de histograma é dada por
$p(r_k) = n_k/M.N$, onde M e N representam o número total de pixels da imagem

In [ ]:
def normalize(histogram, resolution):
    M, N = resolution
    return histogram / (M*N)

In [ ]:
# Cummulative summation of the probabilities

# P.sum() # Summation of the K_(L-1) probabilities
def cumsum(P):
    C = np.zeros_like(P)
    for i in range(C.size):
        cumsum = 0
        for k in range(i+1):
            cumsum += P[k]
        C[i] = cumsum
    return C

In [ ]:
def equalize(img_data=[]):
    if len(img_data) != 2: return
    
    pixels, LMax = img_data
    resolution = pixels.shape
    histogram = h(pixels)
    # Normalize
    P = normalize(histogram, resolution)
    # Sum Probabilities
    C = cumsum(P)
    # Map new indexes
    eq = LMax - 1
    equalized_histogram = np.array(
        np.round(eq * C)
    ).astype('int64')
    # equalize each pixel
    for i in range(pixels.shape[0]):
        for j in range(pixels.shape[1]):
            pixels[i][j] = equalized_histogram[pixels[i][j]]

### Filtro da mediana

In [19]:
# Load Image
header, pixels = load_image('teste.pbm')

In [5]:
def getValue(x, row, col):
    try:
        return x[row][col]
    except IndexError:
        return 0

In [12]:
def median_filter(pixels):    
    # create 3x3 matrix 
    m = np.zeros((3,3))
    
    # create temporary image
    P = np.zeros_like(pixels)

    print('Initializing filter...')
    x, y = pixels.shape
    for i in range(x):
        for j in range(y):
            # first pixel
            m[0][0] = getValue(pixels,i-1,j-1)
            m[0][1] = getValue(pixels,i-1,j)
            m[0][2] = getValue(pixels,i-1,j+1)
            m[1][0] = getValue(pixels,i,j-1)
            # center pixel
            m[1][1] = pixels[i][j]
            m[1][2] = getValue(pixels,i,j+1)
            m[2][0] = getValue(pixels,i+1,j-1)
            m[2][1] = getValue(pixels,i+1,j)
            # last pixel
            m[2][2] = getValue(pixels,i+1,j+1)

            # calculate median value of matrix m
            P[i][j] = math.trunc(np.median(m))
        print(format((i/x),".2%"))
    print("Done.")
    
    return P

In [17]:
# Create filtered image
img = [header, median_filter(pixels)]

Initializing filter...
0.00%
14.29%
28.57%
42.86%
57.14%
71.43%
85.71%
Done.


In [18]:
# Store filtered image
if store_image('teste.pbm', img) == 0: print('Operation Successful!')

Operation Successful!
